# EvaCar

In [325]:
# Import necessary libraries, packages, and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

## 1. Business understanding

### 1.1. Initial analysis

In [326]:
# Load data and create dataframe
df = pd.read_csv("../datasets/RawDataDB.csv")

In [327]:
# Display a sample of data
df.sample(10)

,manufacturer,model,mileage,year,fuel,price,currency,city,county,status,short_description,long_description
10694,Fiat,500,89 000 km,2013,Benzina,8 000,EUR,Bucuresti,Bucuresti,Reactualizat,1.2 8V Collezione,1 242 cm3 • 69 CP • Fiat 500 1.2 benzina Color...
9268,Dacia,Duster,227 923 km,2015,Diesel,7 490,EUR,Otopeni,Bucuresti,Publicat,1.5 dCi 4x4 Ambiance,1 461 cm3 • 109 CP • Dacia Duster / 1.5dCi / 4...
26509,Renault,Laguna,270 000 km,2002,Diesel,2 390,EUR,Iasi,Iasi,Publicat,[],"1 870 cm3 • 120 CP • xenon, piele"
2542,Audi,A6,187 000 km,2017,Diesel,18 900,EUR,Baia,Mare,Publicat,[],1 968 cm3 • 190 CP • audi
22477,Opel,Astra,183 300 km,2017,Benzina,8 989,EUR,Bucuresti,Bucuresti,Reactualizat,1.6 TWINPORT ECOTEC Active Aut.,"1 598 cm3 • 115 CP • 1,6 Benzina Cutie AUTOMAT..."
3336,Audi,Q5,167 500 km,2015,Diesel,17 600,EUR,Oradea,Bihor,Publicat,2.0 TDI Quattro clean,"1 968 cm3 • 190 CP • Al doilea proprietar, mas..."
35291,Volkswagen,Tiguan,195 297 km,2017,Diesel,22 900,EUR,Targoviste,Dambovita,Publicat,2.0 TDI 4Mot DSG Highline,1 968 cm3 • 190 CP • Verificabilă la dealer VW...
25136,Renault,Megane,114 000 km,2015,Diesel,8 900,EUR,Timisoara,Timis,Reactualizat,1.6 dCi GT Line,1 598 cm3 • 130 CP
4641,BMW,X5 xDrive45e,82 052 km,2019,Hibrid,54 849,EUR,Bucuresti,Bucuresti,Reactualizat,[],2 998 cm3 • 286 CP • 46.000e+TVA Plug-in 392CP...
3280,Audi,A4,114 453 km,2017,Diesel,24 997,EUR,Floresti,Cluj,Publicat,Allroad 2.0 TDI Quattro S tronic,1 968 cm3 • 190 CP • / S line / 4x4 / Virtual ...


In [328]:
# Describe 
df.describe(include="all")

,manufacturer,model,mileage,year,fuel,price,currency,city,county,status,short_description,long_description
count,36534,36534,36456,36534.000000,36534,36534,36534,36534,36532,36534,36400,36534
unique,64,1075,13600,NaN,6,6862,2,687,122,2,9676,30012
top,Mercedes-Benz,Passat,1 km,NaN,Diesel,7 990,EUR,Bucuresti,Bucuresti,Publicat,[],1 968 cm3 • 150 CP
freq,4857,1144,704,NaN,22736,178,36499,8696,12848,20101,9024,216
mean,NaN,NaN,NaN,2016.112909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,5.113379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,1966.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,2013.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,2020.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [329]:
# Display information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36534 entries, 0 to 36533
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   manufacturer       36534 non-null  object
 1   model              36534 non-null  object
 2   mileage            36456 non-null  object
 3   year               36534 non-null  int64 
 4   fuel               36534 non-null  object
 5   price              36534 non-null  object
 6   currency           36534 non-null  object
 7   city               36534 non-null  object
 8   county             36532 non-null  object
 9   status             36534 non-null  object
 10  short_description  36400 non-null  object
 11  long_description   36534 non-null  object
dtypes: int64(1), object(11)
memory usage: 3.3+ MB


### 1.2. Process data

In [330]:
# Extract data based on specific element
def extract_data(raw_data: str, splitter: str, specific_element: str) -> str:
    """
    Take a raw elements and return specific part of it
    """
    element = 0
    if type(raw_data) == str:
        raw_element = raw_data.split(f"{splitter}")
        for modified_element in raw_element:
            if element == 0:
                if modified_element.endswith(f"{specific_element}"):
                    element = "".join(modified_element.split(" ")[:-1])
    raw_data = element
    
    return raw_data


# Remove spaces
def remove_space(raw_data: str) -> str:
    """
    Take a raw element and return element with removed white spaces
    """
    raw_data = "".join(raw_data.split(" "))
    
    return raw_data

In [331]:
# Add capacity column
capacity = df["long_description"].apply(extract_data, args=[" • ", "cm3"])
df.insert(loc=3, column="capacity", value=capacity)

In [332]:
# Add power column
power = df["long_description"].apply(extract_data, args=[" • ", "CP"])
df.insert(loc=4, column="power", value=power)

In [333]:
# Delete unnecessary columns
df = df.drop(columns=['short_description', 'long_description'])

In [334]:
# Process mileage column
df["mileage"] = df["mileage"].apply(extract_data, args=[" • ", "km"])

In [335]:
# Remove white spaces from price column
df["price"] = df["price"].apply(remove_space)

In [336]:
# Replace NaN values with zero in mileage column
df['mileage'] = df['mileage'].fillna(0)

In [337]:
# Change data types
data_types = {"mileage": int,
              "capacity": int,
              "power": int,
              "year": int,
              "price": int}

df = df.astype(data_types)

### 1.3. Final analysis

In [338]:
# Display a sample of data
df.sample(10)

,manufacturer,model,mileage,capacity,power,year,fuel,price,currency,city,county,status
162,Audi,A8,103500,2995,340,2018,Benzina,54728,EUR,Cluj-Napoca,Cluj,Reactualizat
25406,Renault,Clio,70796,1461,85,2020,Diesel,12199,EUR,Bucuresti,Bucuresti,Reactualizat
36166,Volvo,S60,199845,1969,150,2016,Diesel,12750,EUR,Bucuresti,Bucuresti,Reactualizat
1124,Audi,Q4,14,0,299,2023,Electric,69000,EUR,Ramnicu,Valcea,Reactualizat
23760,Peugeot,308,272000,1560,92,2013,Diesel,3800,EUR,Slatina,Olt,Reactualizat
33272,Volkswagen,Passat,236830,1968,150,2015,Diesel,14900,EUR,Ploiesti,Prahova,Reactualizat
11141,Ford,Kuga,24000,1996,190,2022,Diesel,27500,EUR,Alexandria,Teleorman,Reactualizat
14193,Hyundai,Veloster,109000,1591,140,2012,Benzina,8799,EUR,Pitesti,Arges,Publicat
32187,Volkswagen,Tiguan,121144,1968,190,2020,Diesel,26989,EUR,Suceava,Suceava,Reactualizat
19442,Mercedes-Benz,GLE Coupe,66400,2925,272,2020,Diesel,89440,EUR,Bucuresti,Bucuresti,Publicat


In [339]:
# Describe 
df.describe(include="all")

,manufacturer,model,mileage,capacity,power,year,fuel,price,currency,city,county,status
count,36534,36534,3.653400e+04,36534.000000,36534.000000,36534.000000,36534,36534.000000,36534,36534,36532,36534
unique,64,1075,NaN,NaN,NaN,NaN,6,NaN,2,687,122,2
top,Mercedes-Benz,Passat,NaN,NaN,NaN,NaN,Diesel,NaN,EUR,Bucuresti,Bucuresti,Publicat
freq,4857,1144,NaN,NaN,NaN,NaN,22736,NaN,36499,8696,12848,20101
mean,NaN,NaN,1.395080e+05,1949.920184,181.762933,2016.112909,NaN,27166.135134,NaN,NaN,NaN,NaN
std,NaN,NaN,9.784439e+04,768.616655,100.162335,5.113379,NaN,35552.730026,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000e+00,0.000000,0.000000,1966.000000,NaN,300.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,6.140000e+04,1498.000000,115.000000,2013.000000,NaN,8857.500000,NaN,NaN,NaN,NaN
50%,NaN,NaN,1.440000e+05,1968.000000,150.000000,2017.000000,NaN,15991.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.060000e+05,1999.000000,204.000000,2020.000000,NaN,30999.000000,NaN,NaN,NaN,NaN


In [340]:
# Display information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36534 entries, 0 to 36533
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   manufacturer  36534 non-null  object
 1   model         36534 non-null  object
 2   mileage       36534 non-null  int32 
 3   capacity      36534 non-null  int32 
 4   power         36534 non-null  int32 
 5   year          36534 non-null  int32 
 6   fuel          36534 non-null  object
 7   price         36534 non-null  int32 
 8   currency      36534 non-null  object
 9   city          36534 non-null  object
 10  county        36532 non-null  object
 11  status        36534 non-null  object
dtypes: int32(5), object(7)
memory usage: 2.6+ MB


In [341]:
# Save processed dataframe to CSV
df.to_csv("../datasets/ProcessedDataDB.csv")